In [32]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
import sklearn.preprocessing         # For scale function
import sklearn.metrics as metrics    # For for label size

In [33]:
#global data
import importlib
import xpuzzle
importlib.reload(xpuzzle)
import node
importlib.reload(node)
import search
importlib.reload(search)
inputData = np.array([])
goalState=np.array([[1,2,3,4], [5,6,7,0]])

In [104]:
#i/o
def noSolution(name, index, data, heu=-1):
    #distance = (2-i)+(2-j) if index == -1 else abs(i-(index/self.state.size))+abs(j-(index%self.state.size))
    # file.write("\ncost: " + str(moves))
    # file.write("\nrunning_time: " + format(time, '.8f'))
    file = open(f"{index}_" + name + heu + '_solution.txt', 'w')
    file.write(data[0])
    file.close()

    file = open(f"{index}_" + name + heu + '_search.txt', 'w')
    file.write(data[1])
    file.close()


def exportData(name, index, data, heu=-1):
    cost = 0
    subname = "-h" + str(heu) if heu > -1 else ""
    solution = f"output/{index}_{name}{subname}" + '_solution.txt'
    search = f"output/{index}_{name}{subname}" + '_search.txt'
    file1 = open(solution, 'w')
    file2 = open(search, 'w')
    if isinstance(p[0], node.Node):
        for ele in data[0].solution:
            file1.write(ele)
        for ele in data[1]:
            file2.write(ele)
    else:  
        file1.write(data[0])
        file2.write(data[1])
    file1.close()
    file2.close()
    

def importData(file_name):
    data = []
    file = open(file_name, 'r')
    for line in file:
        data.append([int(x) for x in line.split(" ")])
    data = np.array(data)
    board_len = len(data[0])
    board_side = int(board_len ** 0.5)
    return data

inputData = importData('samplePuzzles.txt')
print(inputData)

[[3 0 1 4 2 6 5 1]
 [6 3 4 7 1 2 5 0]
 [1 0 3 6 5 2 7 4]
 [0 3 5 1 7 2 4 6]
 [1 3 5 7 0 4 6 2]]


In [110]:
#inputData[0] no solution
index = 3
t=xpuzzle.XPuzzle(2, 4, inputData[index]) 
t.printPuzzle()
s=search.Search(t)


0 3 5 1 
7 2 4 6 



In [111]:
print("A*")
for heuristic in range(2):
    p  = s.aStar(heuristic)
    print(p[0])
    exportData("astar", index, p, heuristic)


A*
ZCRRR
ZCRRR


In [112]:
print("GBFS")
for heuristic in range(2):
    p = s.greedyBFS(heuristic)
    print(p[0])
    exportData("gbfs", index, p, heuristic)


GBFS
ZCRRR
ZCRRR


In [113]:
print("UCS")
p = s.bestFirst()
print(p[0])
exportData("ucs", index, p)

print("\nMoves")
move = str(p[0])
t.parseMoveSequence(move)


UCS
ZCRRR

Moves
1 3 5 0 
7 2 4 6 

1 3 5 7 
0 2 4 6 

1 3 5 7 
2 0 4 6 

1 3 5 7 
2 4 0 6 

1 3 5 7 
2 4 6 0 

